In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

data=pd.read_csv('train_covid.csv')

"""
mean_hh_cli_ct                                            4784 non-null   float64
pct_worried_finances                                      0 non-null      float64
pct_cmnty_cli                                             4804 non-null   float64

"""
#date不是一個特徵值，除非使用OrdinalEncoder，轉換成數列
#如果test的data是未來的這個特徵值，我認為不是一個好的分類指標
#而pct_worried_finances都是NaN可以直接drop掉
data=data.drop(["date","pct_worried_finances"],axis=1)

#分割feature和label
length=data.shape[1]
X=data.iloc[:,0:length-1]
y=data.iloc[:,length-1]

# Data Prepare


## Use SimpleImputer 將nan用中位數替換

In [10]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
#因為前三個欄位為str的type
X_num=X.iloc[:,3:]
X_num=imp_mean.fit_transform(X_num)
X.iloc[:,3:]=X_num
#確認是否nan都替換掉
X.isna().sum().sum()

0

## Using Different oversampler method

In [11]:
from imblearn.over_sampling import SMOTENC

smote_nc = SMOTENC(categorical_features=[0,1,2], random_state=0)
X, y = smote_nc.fit_resample(X, y)

# ros = RandomOverSampler(random_state=0)
# X, y = ros.fit_resample(X, y)


In [12]:
# for i in range(1000,5000,500):  
#     y=data.iloc[i:,length-1]
#     plt.hist(y,bins=10)
#     filename="test_condition_分布隨日期變化\ 最後{}".format(5000-i)
#     plt.savefig(filename) 
#     plt.close()  

## OneHotEncoder

In [13]:
from sklearn.compose import make_column_selector as selector
#將是type 是object的取出來:categorical_columns取出來的index
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

In [15]:
X_cat=X[categorical_columns]

#onehotencoder使用 -> fit -> transform -> return array 
enc =OneHotEncoder(sparse=False)
enc.fit(X_cat)
data_encoded=enc.transform(X_cat)
#產生columns index的名稱
columns_encoded = enc.get_feature_names(categorical_columns)
X_categorical_columns=pd.DataFrame(data_encoded, columns=columns_encoded)

X_withoutcategorical_columns=X.drop(categorical_columns,axis=1)

X=pd.concat([X_withoutcategorical_columns, X_categorical_columns], axis=1)
#onehotencoder之後



In [16]:
X

,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,pct_cmnty_cli,pct_hh_fever,pct_hh_sore_throat,pct_hh_cough,...,state_code_va,state_code_vt,state_code_wa,state_code_wi,state_code_wv,gender_female,gender_male,age_bucket_18-34,age_bucket_35-54,age_bucket_55+
0,691.000000,7.178188e+05,1.943200,2.092700,3.736900,3.043500,16.353709,3.623200,14.658900,25.362300,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,594.000000,2.440767e+06,1.194500,1.023900,2.559700,1.683500,16.353709,2.188600,11.952900,21.212100,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,98.000000,3.051519e+05,0.000000,0.000000,3.191500,1.020400,16.353709,1.020400,8.163300,15.306100,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,759.000000,6.789525e+05,1.992000,1.726400,3.320100,3.425600,16.353709,3.689100,16.754600,25.989400,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,438.000000,8.263480e+05,0.230900,0.230900,1.616600,0.457700,16.353709,0.915300,5.022800,17.123300,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7721,68.212861,2.379435e+05,0.000000,0.337838,0.000000,0.331800,13.090070,0.331800,1.146788,7.724592,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7722,151.822940,6.034892e+05,2.673809,2.673809,4.980876,2.898391,34.417168,2.898391,12.973454,19.491909,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7723,169.627646,3.509105e+05,1.830351,1.830351,6.589123,2.369102,35.602145,3.324040,10.369216,15.746734,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7724,246.479004,5.699816e+05,0.456274,0.421173,2.141070,1.554701,15.890545,1.934649,3.885957,15.309260,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [22]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                       random_state=0)

# __Make Pipeline__

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
pipe = Pipeline([
                    
                 ('scaler', StandardScaler()),
                
                ('min_max', preprocessing.MinMaxScaler(feature_range=(1, 1000))),
                ('feature_select',SelectKBest(chi2)),
                ('clf',clf)
                  ])
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'min_max', 'feature_select', 'clf', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'min_max__clip', 'min_max__copy', 'min_max__feature_range', 'feature_select__k', 'feature_select__score_func', 'clf__base_estimator__ccp_alpha', 'clf__base_estimator__class_weight', 'clf__base_estimator__criterion', 'clf__base_estimator__max_depth', 'clf__base_estimator__max_features', 'clf__base_estimator__max_leaf_nodes', 'clf__base_estimator__min_impurity_decrease', 'clf__base_estimator__min_impurity_split', 'clf__base_estimator__min_samples_leaf', 'clf__base_estimator__min_samples_split', 'clf__base_estimator__min_weight_fraction_leaf', 'clf__base_estimator__random_state', 'clf__base_estimator__splitter', 'clf__base_estimator', 'clf__bootstrap', 'clf__bootstrap_features', 'clf__max_features', 'clf__max_samples', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__verbose', 'clf__warm_start'])

## param_grid setting

In [24]:
from sklearn.model_selection import ShuffleSplit
# 參數篩選
feature_select__k=[100]#50,100,150,200 中
# 
param_grid = { 
    
    'feature_select__k':feature_select__k,
    
    
}
# GridSearchCV : refit==True 會重新用整個data重新訓練一次
gs=GridSearchCV(estimator=pipe,
               param_grid=param_grid,
               scoring='f1_macro',
               cv=ShuffleSplit(n_splits=5, test_size=0.3, random_state=0),
               n_jobs = -1,
              )
gs=gs.fit(X,y)

# Store best_params_


In [25]:
param_grid=gs.best_params_
# type(param_grid)
# for i in param_grid.keys():
#     param_grid[i]=[param_grid[i]]
# print(param_grid)
print(gs.best_params_)
print(gs.best_score_)

df=pd.DataFrame(gs.cv_results_)

# df.to_excel("xgboost_cv_results.xlsx")

{'feature_select__k': 100}
0.8424172195331607


# 輸入test data

In [26]:

test=pd.read_csv("test_covid.csv")

"""
mean_hh_cli_ct                                            4784 non-null   float64
pct_worried_finances                                      0 non-null      float64
pct_cmnty_cli                                             4804 non-null   float64

"""
# test=test.drop(['date', 'state_code', 'gender', 'age_bucket',"mean_hh_cli_ct","pct_worried_finances","pct_cmnty_cli"],axis=1)

test=test.drop(["date","pct_worried_finances"],axis=1)
test.shape

(906, 94)

## Use SimpleImputer 將nan用中位數替換

In [27]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
#因為前三個欄位為str的type
test_num=test.iloc[:,3:]
test_num=imp_mean.fit_transform(test_num)
test.iloc[:,3:]=test_num
#確認是否nan都替換掉
test.isna().sum().sum()

0

## OneHotEncoder  還要再確認是否一致

In [28]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(test)

X_test=test.iloc[:,0:test.shape[1]-1]

#onehotencoder使用 -> fit -> transform -> return array 
enc_test =OneHotEncoder(sparse=False)
enc_test.fit(X_test[categorical_columns])
data_encoded=enc_test.transform(X_test[categorical_columns])

#產生columns index的名稱
columns_encoded = enc_test.get_feature_names(categorical_columns)
X_categorical_columns=pd.DataFrame(data_encoded, columns=columns_encoded)

X_withoutcategorical_columns=X_test.drop(categorical_columns,axis=1)
#為了讓onehotencoder 的種類一樣
X_test=pd.concat([X_withoutcategorical_columns, X_categorical_columns], axis=1)
#onehotencoder之後
X_test.head()
X_test.columns
X_test=pd.DataFrame(X_test,columns=X.columns)
print(X_test.isna().sum().sum())
X_test=X_test.fillna(0)
print(X_test.isna().sum().sum())

5436
0


# test data 訓練

In [30]:
y_test=gs.best_estimator_.predict(X_test)

col=['test_condition']
df=pd.DataFrame(y_test,columns=col)
df.to_csv('BaggingClassifier_oversampling_SMOTENC_ShuffleSplit.csv', index = True,index_label='id') 
#BaggingClassifier_oversampling_SMOTENC_ShuffleSplit : 0.60  0.84


In [59]:
y.shape

(906,)

In [57]:
y_test.shape

(906,)